In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Imports

In [2]:
# !pip install -U torchtext==0.8.0

In [3]:
import torch, torchtext
torch.__version__, torchtext.__version__

('1.8.0+cu101', '0.9.0')

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
 
import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import numpy as np
import spacy
import torchtext
from torchtext.legacy import data
from torchtext.legacy.data import Field, BucketIterator
from tqdm import tqdm as tqdm
import random
import torchtext.vocab as vocab
import time
import pickle
import tokenize
from io import StringIO

# Global Settings¶

In [5]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
 
SEED = 1234
 
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
 
# hyperparams
BATCH_SIZE = 2
learning_rate = 0.0005
N_EPOCHS = 15
CLIP = 1

# Dataset Preprocessing

In [6]:
cd drive/MyDrive/personal_projects/END_CAPSTONE/

/content/drive/.shortcut-targets-by-id/1dbJ7ll0kb1GnK_5zt-JldgU9DD-BmZyU/personal_projects/END_CAPSTONE


In [9]:
with open('./dataset/small_dataset/glove_trained_models/embeddings/pygments_glove_embedding.pkl' , 'rb') as f:
  embedding_corpus = pickle.load(f)
print('shape of the embedding_corpus is ', len(embedding_corpus))
 
with open('./dataset/small_dataset/transformer_dataset/clean_python_dataset_final.pkl' , 'rb') as f:
  python_dataset = pickle.load(f)
print('length of the dataset is ', len(python_dataset))


shape of the embedding_corpus is  667631
length of the dataset is  3399


In [18]:
print('----- testing the format of dataset -----  \n ')
index = random.randint(0,len(python_dataset)-1)
print(python_dataset[0])
print('\n')
print(python_dataset[index]['code'])

----- testing the format of dataset -----  
 
{'obj': 'write a python program to add two numbers', 'code': "num1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\nprint(f'Sum: {sum}')\n"}


def Celsius_To_Fahrenheit(c):
    fahrenheit = (c * 1.8) + 32
    return fahrenheit
11.



##Tokenizer Demo


In [24]:
######### Code is commented #########
# print('python tokenizer demo')
# x = python_dataset[0]['code']
# tokens = tokenize.generate_tokens(StringIO(x).readline)
# for token in tokens:
#     print(token)
######### Comment ends here #########
print(' pygments tokenizer demo')
import pygments
from pygments.lexers import PythonLexer
lexer = PythonLexer()
tokens_test = lexer.get_tokens(python_dataset[index]['code'])
x = [i for i in tokens_test]
print(x[0:5])

 pygments tokenizer demo
[(Token.Keyword, 'def'), (Token.Text, ' '), (Token.Name.Function, 'Celsius_To_Fahrenheit'), (Token.Punctuation, '('), (Token.Name, 'c')]


In [25]:
spacy_en = spacy.load('en')

######### Code is commented #########
## python tokenizer is replaced by pygments tokenizer
# def tokenize_src_code(text):
#     """
#     Tokenizes src code text from a string into a list of words using python tokenize
#     """
#     return [tok for tok in tokenize.generate_tokens(StringIO(text).readline)]
######### Comment ends here #########

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]
                                                    
lexer = PythonLexer()                                                    
def tokenize_src_code_pygments(text):
  tokens_org = lexer.get_tokens(text)
  tokens_filtered = [i[1] for i in tokens_org if i[0] != pygments.token.Comment.Single]
  return tokens_filtered

SRC = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

TRG = Field(tokenize = tokenize_src_code_pygments, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = False, 
            batch_first = True)
 
fields = [('src', SRC),('trg',TRG)]

In [27]:
######### Code is commented #########
# stale_indices = []
# new_python_dataset = []
# for i in range(len(python_dataset)):
#   try:
#     test = tokenize_src_code_pygments(python_dataset[i]['code'])
#     new_python_dataset.append(python_dataset[i])
#   except Exception as e:
#     print(e)
#     stale_indices.append(i)
#     continue
######### Comment ends here #########

In [28]:
# torchtext example
example = [data.Example.fromlist([python_dataset[i]['obj'],python_dataset[i]['code']], fields) for i in tqdm(range(len(python_dataset)))]

100%|██████████| 3399/3399 [00:02<00:00, 1486.98it/s]


### removing example with max_len > 512 
### this is done to remove cuda error 
### because token_emb and pos_emb will have shape mismatch in deocder

In [29]:
for idx,i in enumerate(example):
    if len(i.trg) > 768:
        i.trg = i.trg[:767]
        
max_len = 0
avg_len = 0
count_len = 0
for i in example:
    if len(i.trg)>max_len:
        max_len = len(i.trg)
    if len(i.trg)>512:
        count_len+=1
    avg_len += len(i.trg)
avg_len = avg_len/len(example)
print(max_len, avg_len, count_len)

767 79.83259782288908 31


In [ ]:
# testing data
index = random.randint(0,1000)
example[index].src,example[index].trg

(['73',
  'write',
  'a',
  'python',
  'function',
  'to',
  'print',
  'a',
  'pattern',
  'of',
  'stars',
  '(',
  '*',
  ')'],
 ['for',
  ' ',
  'row',
  ' ',
  'in',
  ' ',
  'range',
  ' ',
  '(',
  '0',
  ',',
  '5',
  ')',
  ':',
  '\n',
  '    ',
  'for',
  ' ',
  'column',
  ' ',
  'in',
  ' ',
  'range',
  ' ',
  '(',
  '0',
  ',',
  ' ',
  'row',
  '+',
  '1',
  ')',
  ':',
  '\n',
  '        ',
  'print',
  ' ',
  '(',
  '"',
  '*',
  '"',
  ',',
  ' ',
  'end',
  '=',
  '"',
  '"',
  ')',
  '\n',
  '    ',
  '\n',
  '    ',
  'print',
  '(',
  "'",
  '\\r',
  "'",
  ')',
  '\n'])

In [30]:
# creating dataset
chatbotDataset = data.Dataset(example, fields)
(train_data, valid_data, test_data) = chatbotDataset.split(split_ratio=[0.70, 0.15, 0.15], random_state=random.seed(SEED))

In [31]:
print('Total training samples --> {}'.format(len(train_data)))
print('Total validation samples --> {}'.format(len(valid_data)))
print('Total test samples --> {}'.format(len(test_data)))

Total training samples --> 2379
Total validation samples --> 510
Total test samples --> 510


In [32]:
# testing data
index = random.randint(0,500)
vars(test_data.examples[index])

{'src': ['write',
  'a',
  'python',
  'program',
  'to',
  'filter',
  'similar',
  'case',
  'strings',
  'and',
  'print',
  'it'],
 'trg': ['x',
  '=',
  '[',
  ']',
  '\n',
  'for',
  ' ',
  'i',
  ' ',
  'in',
  ' ',
  'test_list',
  ':',
  '\n',
  '    ',
  'if',
  ' ',
  'i',
  '.',
  'islower',
  '(',
  ')',
  ' ',
  'or',
  ' ',
  'i',
  '.',
  'isupper',
  '(',
  ')',
  ':',
  '\n',
  '        ',
  'print',
  '(',
  'x',
  '.',
  'append',
  '(',
  'i',
  ')',
  ')',
  '\n',
  'print',
  '(',
  'x',
  ')',
  '\n']}

In [33]:
# build vocab
SRC.build_vocab(train_data, min_freq = 1)
TRG.build_vocab(train_data, min_freq = 1)

In [34]:
######### Code is commented #########
# print('loading and processing python corpus tokens')
# with open('./dataset/small_dataset/python_corpus_tokens_clean_small.pkl', 'rb') as f:
#   python_corpus_tokens_data = pickle.load(f)
 
# import tokenize
# remove_list_temp = ['COMMENT', 'NUMBER' , 'STRING']
# token_names = []
# python_corpus_tokens = []
# for each_token in tqdm(python_corpus_tokens_data):
#   token_name = tokenize.tok_name[each_token.type]
#   if token_name not in token_names:
#     token_names.append(token_name)
#   if token_name not in remove_list_temp:
#     python_corpus_tokens.append(each_token.string)
  
# # python_corpus_tokens = [i.string for i in tqdm(python_corpus_tokens_data)]
# # # removing duplicates
# python_corpus_tokens = list(set(python_corpus_tokens)) 
# print('total number of python tokens ', len(python_corpus_tokens))
 
# word2id = {w:i for i,w in enumerate(python_corpus_tokens)}
# id2word = {i:w for w, i in word2id.items()}
######### Comment ends here #########

### Loading pre-trained glove embedding into target vocab

In [35]:
# TODO try a different methos for this
# https://discuss.pytorch.org/t/aligning-torchtext-vocab-index-to-loaded-embedding-pre-trained-weights/20878/6
sorted_keys = []
not_available = []
embs = []
for each_word in tqdm(TRG.vocab.stoi):
  try:
    selected_id = word2id[each_word]
    sorted_keys.append(selected_id)
    temp_emb = embedding_corpus[each_word]
    embs.append(temp_emb)
  except Exception as e:
    not_available.append(each_word)
    temp_emb = np.random.uniform(low=-1.0, high=1.0, size=(256,)).astype('float32')
    embs.append(temp_emb)
pretrained_embs = torch.from_numpy(np.array(embs))
TRG.vocab.vectors = pretrained_embs

100%|██████████| 4665/4665 [00:00<00:00, 121522.31it/s]


In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.src),
    sort_within_batch=True,
    device = device)

In [37]:
print('checking out one batch')
x = next(iter(train_iterator))
print(x.src.shape,x.trg.shape)

checking out one batch
torch.Size([2, 13]) torch.Size([2, 60])


In [38]:
'''
    Encoder arguments
        input_dim --> [src_len,batch_size]
        hid_dim --> convert each input word to embedding
        n_layers --> number of transformer layers
        n_heads --> number of heads for multi head attention
        pf_dim --> hidden layer upscaling in pointwise feedforward layer
        dropout --> dropout value 
        device --> gpu or cpu
        max_length --> max langth of the sentence, used in positional encoding
        
    Encoder outputs
        src after changing it through the self attention and pointwise feed forward layer
        
    Encoder description
        - combines source and positional embedding
        - passes it through self attention layer and calculates attention
        - passes attention output to pointwise feed forward layer 
        - returns the src after passing it to the aforementioned layers
'''
 
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()
 
        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [39]:
'''
    EncoderLayer arguments
    
        hid_dim --> convert each input word to embedding
        n_heads --> number of heads for multi head attention
        pf_dim --> hidden layer upscaling in pointwise feedforward layer
        dropout --> dropout value 
        device --> gpu or cpu
        
    EncoderLayer outputs
        src after changing it through the self attention and pointwise feed forward layer
        
    EncoderLayer description
        -  First It takes the src coming from encoder and passes it through attention mechanism.
        -  Then applies the layer norm and feed forward operations on the transformed source
'''
 
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [40]:
'''
    MultiHeadAttentionLayer arguments
    
        hid_dim --> convert each input word to embedding
        n_heads --> number of heads for multi head attention
        dropout --> dropout value 
        device --> gpu or cpu
        
    MultiHeadAttentionLayer outputs
        src after changing it through the self attention mechanism and the attention mask
        
    MultiHeadAttentionLayer description
        -  takes the query key and value matrices
        -  computes energy by multiplying query and key matrices
        -  apply mask on the energy vector
        -  while processing the encoder src mask is the mask where the unnecessary pad tokens are removed
        -  while processing the decoder trg mask is the mask where future information is masked so that
        -  decoder can only see previous and current word while decoding next word
        -  the energy matrix is then multiplied by the value matrix to get the final transformed source matrix
        -  we also return the attention mask after applying a softmax on the energy vector 
        -  this could later be used to visually test the attention on words
'''
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [41]:
'''
    PositionwiseFeedforwardLayer arguments
    
        hid_dim --> convert each input word to embedding
        pf_dim --> hidden layer upscaling in pointwise feedforward layer
        dropout --> dropout value 
        
    PositionwiseFeedforwardLayer outputs
        src after changing it through couple of linear layers and relu activation
        
'''
 
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [42]:
'''
    Decoder arguments
        output_dim --> [teg_len,batch_size]
        hid_dim --> convert each input word to embedding
        n_layers --> number of transformer layers
        n_heads --> number of heads for multi head attention
        pf_dim --> 
        dropout --> dropout value 
        device --> gpu or cpu
        max_length --> max langth of the sentence, used in positional encoding
        
    Decoder outputs
        trg after changing it through the layers shown in the figure above
        
    Decoder description
        - combines trg and postional embedding
        - passes it through the decoder layers as shown in the figure 
        - the final output is passed through linear layer to match output vocab dimension
        - we would later do softmax on this output to find the best word prediction
 
'''
 
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 1024):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
 
        weight_matrix = TRG.vocab.vectors
        self.tok_embedding.weight.data.copy_(weight_matrix )
 
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

In [43]:
'''
    DecoderLayer arguments
    
        hid_dim --> convert each input word to embedding
        n_heads --> number of heads for multi head attention
        pf_dim --> hidden layer upscaling in pointwise feedforward layer
        dropout --> dropout value 
        device --> gpu or cpu
        
    DecoderLayer outputs
        trg after changing it through the self attention and pointwise feed forward layers
        
    DecoderLayer description
        - The decoder layer is almost as same as encoder layer with thr following changes
        - decoder layer uses 2 attention layers
        - first trg goes through self attention layer and attention is calculated, here trg is masked using trg_mask
        - attention output is then passed through add and layernorm block
        - again we calculate attention using encoder key value and decoder query
        - this 2nd attention output is then passed through add and layernorm block
'''
 
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        # query, key, value
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

In [44]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
 
        #src_mask = [batch size, 1, 1, src len]
 
        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask
 
    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [45]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1
 
enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)
 
dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

In [46]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
 
model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [47]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
 
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 7,067,449 trainable parameters


In [48]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [49]:
model.apply(initialize_weights);

In [50]:
LEARNING_RATE = 0.0005
 
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [51]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [52]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [53]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):
 
            src = batch.src
            trg = batch.trg
 
            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)
 
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [54]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [55]:
best_valid_loss = float('inf')
N_EPOCHS = 15
for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        # torch.save(model.state_dict(), './dataset/small_dataset/trained_srcgen_model/srcgen_chatbot.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 37s
	Train Loss: 3.098 | Train PPL:  22.156
	 Val. Loss: 2.565 |  Val. PPL:  12.995
Epoch: 02 | Time: 0m 36s
	Train Loss: 2.366 | Train PPL:  10.657
	 Val. Loss: 2.281 |  Val. PPL:   9.784
Epoch: 03 | Time: 0m 36s
	Train Loss: 2.071 | Train PPL:   7.937
	 Val. Loss: 2.134 |  Val. PPL:   8.447
Epoch: 04 | Time: 0m 36s
	Train Loss: 1.851 | Train PPL:   6.364
	 Val. Loss: 1.980 |  Val. PPL:   7.243
Epoch: 05 | Time: 0m 36s
	Train Loss: 1.663 | Train PPL:   5.274
	 Val. Loss: 1.876 |  Val. PPL:   6.530
Epoch: 06 | Time: 0m 36s
	Train Loss: 1.500 | Train PPL:   4.483
	 Val. Loss: 1.827 |  Val. PPL:   6.218
Epoch: 07 | Time: 0m 36s
	Train Loss: 1.366 | Train PPL:   3.919
	 Val. Loss: 1.774 |  Val. PPL:   5.897
Epoch: 08 | Time: 0m 36s
	Train Loss: 1.257 | Train PPL:   3.516
	 Val. Loss: 1.746 |  Val. PPL:   5.733
Epoch: 09 | Time: 0m 36s
	Train Loss: 1.160 | Train PPL:   3.189
	 Val. Loss: 1.710 |  Val. PPL:   5.529
Epoch: 10 | Time: 0m 36s
	Train Loss: 1.083 | Train PPL

In [56]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):
    
    model.eval()
        
    tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [57]:
# test on one exmaple
def chatting(example_idx):
    src = vars(test_data.examples[example_idx])['src']
    trg = vars(test_data.examples[example_idx])['trg']

    src_sen = ' '.join(src)
    trg_sen = ' '.join(trg)
    print('---------------------------------------')
    print(f'Actual src is ==> \n {src_sen}')
    print(f'Actual trg is ==> \n {trg_sen}')

    translation, attention = translate_sentence(src, SRC, TRG, model, device)

    translation_sen = translation[:-1]
    translation_sen = ' '.join(translation_sen)
    print(f'Predicted trg is ==>\n {translation_sen}')
    print('---------------------------------------')
    return src_sen,trg_sen,translation[:-1]

In [58]:
example_idx = random.randint(0,len(test_data))
src,trg,translation = chatting(example_idx)

---------------------------------------
Actual src is ==> 
 write a function to return the speed of moving object based of distance travelled in given time
Actual trg is ==> 
 def   cal_speed ( distance : float , time : float ) - > float : 
      return   distance / time 

Predicted trg is ==>
 def   cal_density ( mass / volume ) : 
      return   ( mass / volume ) 

---------------------------------------


In [59]:
example_idx = random.randint(0,len(test_data))
src,trg,translation = chatting(example_idx)

---------------------------------------
Actual src is ==> 
 write a python program to print the even numbers from a given list
Actual trg is ==> 
 def   is_even_num ( l ) : 
 enum   =   [ ] 
 for   n   in   l : 
 if   n   %   2   ==   0 : 
 enum . append ( n ) 
 return   enum 
 print ( is_even_num ( [ 1 ,   2 ,   3 ,   4 ,   5 ,   6 ,   7 ,   8 ,   9 ] ) ) 

Predicted trg is ==>
 list1   =   [ 11 ,   - 21 ,   0 ,   0 ,   0 ,   - 45 ,   66 ,   - 93 ] 
 for   num   in   list1 : 
      if   num   <   0
---------------------------------------


In [61]:
example_idx = random.randint(0,len(test_data))
src,trg,translation = chatting(example_idx)

---------------------------------------
Actual src is ==> 
 write a python function to multiply all values in a list
Actual trg is ==> 
 def   multiplyList ( myList )   : 
      result   =   1 
      for   x   in   myList : 
          result   =   result   *   x 
      return   result 

Predicted trg is ==>
 def   reverse_string ( str_to_be_reversed ) : 
      return   str_to_be_reversed [ : : : - 1 ] 

---------------------------------------


In [ ]:
# def tokenize_src_code_test(text):
#     """
#     Tokenizes src code text from a string into a list of words using python tokenize
#     """
#     return [tok for tok in tokenize.generate_tokens(StringIO(text).readline)]

# tokenize.untokenize(tokenize_src_code_test(trg))

In [ ]:
#todo
# 1. check the embedding weight copy part
# 2. check the max_len part that was showing cuda error
# 3. save pretrained embedding in vector format
# 4. save output to text file
# 5. https://github.com/bentrevett/pytorch-sentiment-analysis/blob/69fc4b0c6952247c33dc9fc7f09123dbd1ef023b/C%20-%20Loading%2C%20Saving%20and%20Freezing%20Embeddings.ipynb